In [ ]:
import pandas as pd 
import numpy as np
from os import listdir
from os.path import isfile, join
import plotly.express as px
import pytz
eastern = pytz.timezone('US/Eastern')

from dotmap import DotMap
import yaml
config = DotMap(yaml.safe_load(open("src/config.yaml")), _dynamic=False)

In [ ]:
def filter_trading_hours(df, time_column):
    T = df[time_column].dt
    min_mask = (T.hour >= 10) | ((T.hour == 9) & (T.minute >= 31))
    max_mask = (T.hour < 16) | ((T.hour == 16) & (T.minute <= 1))
    return df.loc[min_mask & max_mask, :]

In [ ]:
def preprocess_iq_feed_prices(prices: pd.DataFrame) -> pd.DataFrame: 
    if "time" in prices.columns:
        # Intra-day data
        prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(None)
        prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(eastern)
        prices.drop_duplicates(keep="first", inplace=True)
        prices.dropna(inplace=True)
        
        prices = filter_trading_hours(df=prices, time_column="time")

        # Deals with duplicate rows which occurr when not all the digits for volume are 
        # correctly entered, but only the first 1-3. So keep the largest.
        prices = prices.sort_values(["time", "volume"], ascending=[True, False])
        prices = prices.drop_duplicates(subset=["time"], keep="first")

        prices.set_index("time", inplace=True)
        prices.sort_index(ascending=True, inplace=True)
        assert prices.index.is_unique
    else:
        # Daily data
        prices.dropna(inplace=True)
        prices["date"] = pd.to_datetime(prices.date)
    return prices

In [ ]:
onlyfiles = [f for f in listdir(config.data.iqfeed.minute.raw) if isfile(join(config.data.iqfeed.minute.raw, f))]
tickers = [x.split("_")[0] for x in onlyfiles]
ticker = "GOOGL"

In [ ]:
prices: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/{ticker}_1min.parquet", 
                                                                 columns=["time", "close", "open", "volume"]))
prices_daily: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/daily/{ticker}_daily.parquet", 
                                                                       columns=["date", "close", "open", "volume"]))

# Adjust for Splits and Dividends

In [ ]:

intra_opens = prices[(prices.index.hour == 9) & (prices.index.minute == 0), "open"]
intra_opens.index = intra_opens.index.date
intra_opens.index.name = "date"

intra_closes = prices[(prices.index.hour == 16) & (prices.index.minute == 1), "open"]
intra_closes.index = intra_closes.index.date
intra_closes.index.name = "date"
intra_closes.name = "intra_close"

In [ ]:
merged = pd.merge(intra_opens, prices_daily["date", "close", "open"], left_on="date", right_on="date", suffixes=("_intra", "_eod"))
merged = pd.merge(merged, intra_closes, left_on="date", right_on="date")

In [ ]:
merged = merged.sort_index(ascending=False)
merged["split_ratio"] = 1

In [ ]:
# Calculate Dividend and Split Events (They are handled exactly the same, but dividend events are smaller)
# To adjust, we go backwrds in time. 
# When a split has occurred the close will be the first price that is influenced in the historical adjusted time series.
# Assuming the split/dividend has occurred overnight.
for i in range(len(merged.index)):
    row = merged.index[i]
    
    split_ratios = merged.iloc[i:i+5]["close_eod"] / merged.iloc[i:i+5]["close_intra"]
    
    # All split ratios are larger than 0.5% => persistent discrepancy => split/dividend event
    if min(np.abs(split_ratios - 1)) >= 0.005:
        split_ratio = merged[row, "close_eod"] / merged[row, "close_intra"]
        merged[row:, ["close_intra", "split_ratio"]]  = [merged[row:, "close_intra"] * split_ratio, "split_ratio"]

In [ ]:
prices["split_ratio"] = 1
prices[merged.index, "split_ratio"] = merged["split_ratio"]
prices["cum_split_ratio"] = np.cumprod(prices["split_ratio"][::-1])[::-1] 

In [ ]:
# split_ratio < 1 => price goes down
# stocks dont trade in fractions of a penny, hence we need to round to 2 decimals
prices[["open", "high", "low", "close"]] = (prices[["open", "high", "low", "close"]] * prices["cum_split_ratio"]).round(decimals=2)

# split_ratio < 1 => volume goes up, since price goes down and pricevolume has to stay the same 
# (volume denotes number of stocks traded)
prices["volume"] = prices["volume"] / prices["cum_split_ratio"]

In [ ]:
px.line(merged["close_intra", "close_eod"])

# Check quality looking at the amount of splits/dividends

In [ ]:
# Download yfinance split data and check if there is a difference between the calculated splits/dividends
# IMPORTANT !!!!!!!!!!
# In yahoo finance the `Close` is adj. for splits and the `Adj. Close` for splits and dividends
# Dividends on Yahoo Finance are adjusted for splits!
import yfinance

df=yfinance.download("AAPL",period="10y",actions=True)

In [ ]:
df["split_ratio"] = 1
# split_mask = df["Stock Splits"] > 0
# df.loc[split_mask, "split_ratio"] = 1 / df.loc[split_mask, "Stock Splits"] # Do we have to apply the split the next day or the current day?

df["Dividends"] = df["Dividends"].shift(-1, fill_value=0)
dividend_mask = (df["Dividends"] > 0)
df.loc[dividend_mask, "split_ratio"] = 1 - df.loc[dividend_mask, "Dividends"] / df.loc[dividend_mask, "Close"]

In [ ]:
df["cum_split_ratio"] = np.cumprod(df["split_ratio"][::-1])[::-1] 
df["Close"] = df["Close"] * df["cum_split_ratio"]

In [ ]:
df

In [ ]:
# Check quality looking at the difference between intra_day adj open and eod adj open.
# This difference should be smaller than the split_ratio for all days where the split_ratio is not 1
# This difference should also be smaller in general than our threshold for splits (1%?)

In [ ]:
# Once this has been checked and intraday data consistency/continuity is assured we could
# Replace all the closes and opens from the intra day data set with those of the eod data set.
# However this leads to problems, if we calculate wrong a single split event.
# If we don't do this replacement only one bday is affected (split day). 
# If we do the replacement multiple days will be affected by the error.

# Save adjusted files to disk